In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [7]:
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt
import numpy as np
import cv2
from tqdm import tqdm
import os

In [8]:
# import train_csv

import pandas as pd
classes = pd.read_csv('../input/planets-dataset/planet/planet/train_classes.csv')

In [9]:
#split the various tags in train_csv
classes['splitted tags'] = classes['tags'].apply(lambda x: x.split())


In [10]:
# combine all tags into a set to know the unique number of tags
combine_list = []
for i, n in enumerate(classes['splitted tags']):
    combine_list.extend(n)
combine_set = set(combine_list) 
len(combine_set)

17

In [11]:
# visualize how frequent the different labels occur

# from collections import Counter
# count_dic = Counter(combine_list)
# count_df = pd.DataFrame(count_dic, index = ['counts']).T
# plt.figure(figsize=(10, 8))
# sns.barplot(x=count_df.index, y= count_df['counts'])
# plt.xticks(rotation= 45, fontsize = 13)

In [12]:
# building the train_labels
columns = sorted(list(combine_set))
train_labels = pd.DataFrame(np.zeros((40479, 17)), columns = columns)
train_labels

,agriculture,artisinal_mine,bare_ground,blooming,blow_down,clear,cloudy,conventional_mine,cultivation,habitation,haze,partly_cloudy,primary,road,selective_logging,slash_burn,water
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40474,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40475,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40476,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40477,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
for n, lists in tqdm(enumerate(classes['splitted tags'])):
    for column in lists:
        train_labels[column][n]=1


40479it [00:28, 1407.43it/s]


In [14]:
train_labels['image_name'] = classes['image_name'] + '.jpg'
train_labels

,agriculture,artisinal_mine,bare_ground,blooming,blow_down,clear,cloudy,conventional_mine,cultivation,habitation,haze,partly_cloudy,primary,road,selective_logging,slash_burn,water,image_name
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,train_0.jpg
1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,train_1.jpg
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,train_2.jpg
3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,train_3.jpg
4,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,train_4.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40474,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,train_40474.jpg
40475,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,train_40475.jpg
40476,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,train_40476.jpg
40477,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,train_40477.jpg


In [15]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD, Adam
from keras.applications import VGG16
from keras.models import Model
from keras.callbacks import ReduceLROnPlateau

In [16]:
# defining the metric value function
from keras import backend

def fbeta(y_true, y_pred, beta=2):
    #clip predictions
    y_pred = backend.clip(y_pred, 0, 1)
    # calculate elements
    tp = backend.sum(backend.round(backend.clip(y_true * y_pred, 0, 1)), axis = 1)
    fp = backend.sum(backend.round(backend.clip(y_true - y_pred, 0, 1)), axis = 1)
    fn = backend.sum(backend.round(backend.clip(y_pred - y_true, 0, 1)), axis = 1)
    
    # calculate precision
    p = tp/(tp + fp + backend.epsilon())
    #calculate recall
    r = tp / (tp + fn + backend.epsilon())
    #calculate fbeta
    bb = beta ** 2
    fbeta_score = backend.mean((1+ bb) * (p * r)/ (bb * p + r + backend.epsilon()))
    return fbeta_score

In [17]:
train_datagen = ImageDataGenerator(featurewise_center = True, 
                                   horizontal_flip = True,
                                   vertical_flip = True,
                                   rotation_range = 90)
val_datagen = ImageDataGenerator(featurewise_center = True)
train_datagen.mean = [123.68, 116.779, 103.939]
val_datagen.mean= [123.68, 116.779, 103.939]


In [18]:
columns

['agriculture',
 'artisinal_mine',
 'bare_ground',
 'blooming',
 'blow_down',
 'clear',
 'cloudy',
 'conventional_mine',
 'cultivation',
 'habitation',
 'haze',
 'partly_cloudy',
 'primary',
 'road',
 'selective_logging',
 'slash_burn',
 'water']

In [19]:
# creating a scaled iterator ti feed into model
train_it = train_datagen.flow_from_dataframe(train_labels[: 30479], 
                                             directory = '../input/planets-dataset/planet/planet/train-jpg', 
                                             x_col = 'image_name', y_col = columns, 
                                             class_mode= 'raw', batch_size = 64, target_size = (128, 128))
val_it = val_datagen.flow_from_dataframe(train_labels[30479:], 
                          directory = '../input/planets-dataset/planet/planet/train-jpg', 
                          x_col = 'image_name', 
                          y_col = columns,
                          class_mode= 'raw', 
                          batch_size = 64, target_size = (128, 128))


Found 30479 validated image filenames.
Found 10000 validated image filenames.


In [20]:
len(train_it)

477

In [21]:
import gc
gc.collect()

40

In [22]:

# model = Sequential()
# model.add(Conv2D(32, (3,3), padding = 'same', input_shape=(128, 128, 3)))
# model.add(Conv2D(32, (3,3), padding = 'same'))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2,2)))

# model.add(Conv2D(64, (3,3), padding='same'))
# model.add(Conv2D(64, (3,3), padding='same'))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2,2)))


# model.add(Conv2D(128, (3,3), padding='same'))
# model.add(Conv2D(128, (3,3), padding='same'))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2,2)))


# model.add(Flatten())

# model.add(Dense(128))
# model.add(Activation('relu'))

# model.add(Dense(17))
# model.add(Activation('sigmoid'))
# opt = SGD(lr=0.01, momentum=0.9)
# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[fbeta])

# history = model.fit_generator(train_it, steps_per_epoch = len(train_it), epochs = 30, 
#                               validation_data= val_it, validation_steps = len(val_it))

In [23]:
# load model
model = VGG16(include_top = False, input_shape= (128, 128, 3))

# define droputs
dropout1 = Dropout(0.2)
dropout2 = Dropout(0.2)

# mark loaded layers as not trainable
for layer in model.layers:
    layer.trainable = False
    
# allow las vgg block to be trainable
model.get_layer('block5_conv1').trainable = True
model.get_layer('block5_conv2').trainable = True
model.get_layer('block5_conv3').trainable = True
model.get_layer('block5_pool').trainable= True

# add new classifier layers
conv_layer2 = Conv2D(64, (3,3), padding='same', activation = 'relu')(model.layers[-1].output)
conv_layer3 = Conv2D(64, (3,3), padding='same', activation = 'relu')(conv_layer2)
pool_layer2 = MaxPooling2D(pool_size=(2,2))(conv_layer3)
x = dropout1(pool_layer2)
flat1 = Flatten()(x)
class1 = Dense(128, activation='relu')(flat1)
class_dropout = dropout2(class1)
output = Dense(17, activation='sigmoid')(class_dropout)

# define new model
model = Model(inputs=model.inputs, outputs=output)
opt = Adam(lr=0.001)
reduc_lr = ReduceLROnPlateau(monitor = 'val_loss', factor= 0.2, patience=5, min_lr = 0.001)
# compile model
model.compile(optimizer= 'adam', loss = 'binary_crossentropy', metrics = [fbeta])


58892288/58889256 [==============================] - 1s 0us/step


In [25]:
history = model.fit_generator(train_it, steps_per_epoch = len(train_it), validation_data = val_it, 
                             validation_steps = len(val_it), epochs = 25, callbacks = [reduc_lr])

Epoch 1/25
477/477 [==============================] - 285s 598ms/step - loss: 0.2042 - fbeta: 0.7950 - val_loss: 0.1514 - val_fbeta: 0.8556
Epoch 2/25
477/477 [==============================] - 222s 465ms/step - loss: 0.1527 - fbeta: 0.8627 - val_loss: 0.1362 - val_fbeta: 0.8786
Epoch 3/25
477/477 [==============================] - 226s 474ms/step - loss: 0.1384 - fbeta: 0.8786 - val_loss: 0.1257 - val_fbeta: 0.8909
Epoch 4/25
477/477 [==============================] - 230s 483ms/step - loss: 0.1329 - fbeta: 0.8838 - val_loss: 0.1230 - val_fbeta: 0.8915
Epoch 5/25
477/477 [==============================] - 230s 481ms/step - loss: 0.1285 - fbeta: 0.8878 - val_loss: 0.1185 - val_fbeta: 0.8957
Epoch 6/25
477/477 [==============================] - 228s 478ms/step - loss: 0.1262 - fbeta: 0.8893 - val_loss: 0.1140 - val_fbeta: 0.9002
Epoch 7/25
477/477 [==============================] - 230s 483ms/step - loss: 0.1233 - fbeta: 0.8920 - val_loss: 0.1160 - val_fbeta: 0.8960
Epoch 8/25
477/477 [

In [ ]:
model.save('semifinal_model.h5')

In [26]:
import gc
gc.collect()

47

In [27]:
import numpy as np
test_images = []
for img1 in os.listdir('../input/planets-dataset/planet/planet/test-jpg'):
    test_images.append('../input/planets-dataset/planet/planet/test-jpg/{}'.format(img1))
for image in os.listdir('../input/planets-dataset/test-jpg-additional/test-jpg-additional'):
    test_images.append('../input/planets-dataset/test-jpg-additional/test-jpg-additional/{}'.format(image))

In [28]:
test_labels = pd.DataFrame(test_images, columns = ['image_names'])
test_labels

,image_names
0,../input/planets-dataset/planet/planet/test-jp...
1,../input/planets-dataset/planet/planet/test-jp...
2,../input/planets-dataset/planet/planet/test-jp...
3,../input/planets-dataset/planet/planet/test-jp...
4,../input/planets-dataset/planet/planet/test-jp...
...,...
61186,../input/planets-dataset/test-jpg-additional/t...
61187,../input/planets-dataset/test-jpg-additional/t...
61188,../input/planets-dataset/test-jpg-additional/t...
61189,../input/planets-dataset/test-jpg-additional/t...


In [29]:
test_datagen = ImageDataGenerator(featurewise_center= True)
test_datagen.mean = [123.68, 116.779, 103.939]
test_generator= test_datagen.flow_from_dataframe(test_labels, directory = None,
                                                 x_col = 'image_names', 
                                                 y_col = None, batch_size = 64, 
                                                 class_mode=None, target_size=(128, 128), 
                                                 shuffle = False)

Found 61191 validated image filenames.


In [37]:
test_generator.reset()
pred = model.predict_generator(test_generator, steps = len(test_generator))

In [38]:
predictions = pd.DataFrame(pred, columns = columns)

In [36]:
predictions

,agriculture,artisinal_mine,bare_ground,blooming,blow_down,clear,cloudy,conventional_mine,cultivation,habitation,haze,partly_cloudy,primary,road,selective_logging,slash_burn,water,image_name
0,0.087565,9.199724e-06,0.000783,2.496328e-02,8.499561e-03,0.305994,2.127982e-06,6.354824e-07,0.127438,0.011277,0.000901,0.739373,0.999990,0.015643,0.007944,6.633467e-04,0.018757,test_19444.jpg
1,0.010013,2.396973e-06,0.000077,5.092846e-03,7.941002e-04,0.268129,8.456128e-03,1.010262e-08,0.006832,0.001442,0.143544,0.430784,0.993396,0.002159,0.000670,8.070952e-06,0.006442,test_6644.jpg
2,0.024360,1.012276e-07,0.000105,4.660099e-06,1.682574e-06,0.000459,7.428090e-02,2.216773e-08,0.008038,0.000532,0.001009,0.965230,0.888927,0.005367,0.000024,2.650797e-06,0.031735,test_19524.jpg
3,0.207750,9.426285e-05,0.014865,2.048879e-04,2.224830e-04,0.001395,7.056131e-02,2.053929e-04,0.067427,0.009107,0.001699,0.968566,0.907820,0.051905,0.001031,1.347829e-03,0.138727,test_16697.jpg
4,0.006609,2.704654e-08,0.000031,1.346944e-02,1.245619e-03,0.998686,1.756035e-09,1.104959e-10,0.008300,0.001096,0.000131,0.000741,1.000000,0.001301,0.000817,6.714528e-07,0.001731,test_20036.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61186,0.003925,2.028368e-08,0.000014,5.068430e-02,1.298491e-03,0.999840,4.649519e-12,4.840347e-11,0.006784,0.001500,0.000003,0.000240,1.000000,0.002311,0.003745,4.661889e-07,0.001236,file_6688.jpg
61187,0.037991,5.056921e-07,0.000320,3.594403e-07,5.818293e-08,0.005883,6.662351e-01,5.091860e-08,0.005997,0.002564,0.016669,0.326233,0.311728,0.008990,0.000002,2.625011e-07,0.058328,file_15179.jpg
61188,0.990274,1.702297e-05,0.011974,1.787372e-06,8.287532e-07,0.969539,3.356980e-09,1.777838e-04,0.206631,0.325898,0.010835,0.019997,0.992668,0.898041,0.000078,6.933459e-03,0.090454,file_8403.jpg
61189,0.200814,2.786894e-04,0.012900,1.182579e-03,1.493002e-04,0.979906,1.917700e-06,2.327780e-04,0.070137,0.024870,0.017459,0.005630,0.996980,0.103034,0.001599,6.650184e-04,0.967168,file_1385.jpg


In [33]:
def get_columns_ones(df, n):
    d = dict(zip(df.columns, list(df.iloc[n])))
    # extract keys of dict with ones
    l = [k for k, v in d.items() if v >= 0.2]
    result = ' '.join(l)
    return result

In [39]:
predictions2 = predictions.copy()
predictions2['tags'] = 0
for i in predictions2.index:
    predictions2['tags'][i] = get_columns_ones(predictions, i)
predictions2

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


,agriculture,artisinal_mine,bare_ground,blooming,blow_down,clear,cloudy,conventional_mine,cultivation,habitation,haze,partly_cloudy,primary,road,selective_logging,slash_burn,water,tags
0,0.087565,9.199724e-06,0.000783,2.496328e-02,8.499561e-03,0.305994,2.127982e-06,6.354824e-07,0.127438,0.011277,0.000901,0.739373,0.999990,0.015643,0.007944,6.633467e-04,0.018757,clear partly_cloudy primary
1,0.010013,2.396973e-06,0.000077,5.092846e-03,7.941002e-04,0.268129,8.456128e-03,1.010262e-08,0.006832,0.001442,0.143544,0.430784,0.993396,0.002159,0.000670,8.070952e-06,0.006442,clear partly_cloudy primary
2,0.024360,1.012276e-07,0.000105,4.660099e-06,1.682574e-06,0.000459,7.428090e-02,2.216773e-08,0.008038,0.000532,0.001009,0.965230,0.888927,0.005367,0.000024,2.650797e-06,0.031735,partly_cloudy primary
3,0.207750,9.426285e-05,0.014865,2.048879e-04,2.224830e-04,0.001395,7.056131e-02,2.053929e-04,0.067427,0.009107,0.001699,0.968566,0.907820,0.051905,0.001031,1.347829e-03,0.138727,agriculture partly_cloudy primary
4,0.006609,2.704654e-08,0.000031,1.346944e-02,1.245619e-03,0.998686,1.756035e-09,1.104959e-10,0.008300,0.001096,0.000131,0.000741,1.000000,0.001301,0.000817,6.714528e-07,0.001731,clear primary
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61186,0.003925,2.028368e-08,0.000014,5.068430e-02,1.298491e-03,0.999840,4.649519e-12,4.840347e-11,0.006784,0.001500,0.000003,0.000240,1.000000,0.002311,0.003745,4.661889e-07,0.001236,clear primary
61187,0.037991,5.056921e-07,0.000320,3.594403e-07,5.818293e-08,0.005883,6.662351e-01,5.091860e-08,0.005997,0.002564,0.016669,0.326233,0.311728,0.008990,0.000002,2.625011e-07,0.058328,cloudy partly_cloudy primary
61188,0.990274,1.702297e-05,0.011974,1.787372e-06,8.287532e-07,0.969539,3.356980e-09,1.777838e-04,0.206631,0.325898,0.010835,0.019997,0.992668,0.898041,0.000078,6.933459e-03,0.090454,agriculture clear cultivation habitation prima...
61189,0.200814,2.786894e-04,0.012900,1.182579e-03,1.493002e-04,0.979906,1.917700e-06,2.327780e-04,0.070137,0.024870,0.017459,0.005630,0.996980,0.103034,0.001599,6.650184e-04,0.967168,agriculture clear primary water


In [40]:
predictions2['image_name'] = test_labels['image_names'].apply(lambda x: x.split('/')[-1])

In [41]:
predictions2 = predictions2[['image_name', 'tags']]

In [42]:
predictions2['image_name'] = predictions2['image_name'].apply(lambda x: x.split('.')[0])
predictions2

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,image_name,tags
0,test_19444,clear partly_cloudy primary
1,test_6644,clear partly_cloudy primary
2,test_19524,partly_cloudy primary
3,test_16697,agriculture partly_cloudy primary
4,test_20036,clear primary
...,...,...
61186,file_6688,clear primary
61187,file_15179,cloudy partly_cloudy primary
61188,file_8403,agriculture clear cultivation habitation prima...
61189,file_1385,agriculture clear primary water


In [ ]:
submission = pd.read_csv('../input/planets-dataset/planet/planet/sample_submission.csv')


In [ ]:
new_submissions = pd.merge(submission, predictions2, on='image_name', how='left')

In [ ]:
new_submissions.columns = ['image_name', 'remove', 'tags']
new_submission = new_submissions.drop(columns= 'remove')

In [ ]:
new_submission.to_csv('submission.csv', index=False)

![img1](third.png)

![img2](fourth.png)